In [1]:
!pip install pandas numpy scikit-learn xgboost


In [2]:
import pandas as pd

df = pd.read_csv(r"D:\Ecommerce_Tableau\customer360_gold.csv")
df.head()


,ID,YEAR_BIRTH,AGE,EDUCATION,MARITAL_STATUS,INCOME,TOTAL_CHILDREN,TOTAL_SPENT,TOTAL_PURCHASES,WEB_VISITS,RECENCY,CAMPAIGN_RESPONSE,DT_CUSTOMER
0,5524,1957,68,GRADUATION,SINGLE,58138.0,0,1617,25,7,58,1,2012-09-04
1,2174,1954,71,GRADUATION,SINGLE,46344.0,2,27,6,5,38,0,2014-03-08
2,4141,1965,60,GRADUATION,TOGETHER,71613.0,0,776,21,4,26,0,2013-08-21
3,6182,1984,41,GRADUATION,TOGETHER,26646.0,1,53,8,6,26,0,2014-02-10
4,5324,1981,44,PHD,MARRIED,58293.0,1,422,19,5,94,0,2014-01-19


In [3]:
df = df.fillna(0)


In [4]:
X = df.drop(["CAMPAIGN_RESPONSE"], axis=1)
y = df["CAMPAIGN_RESPONSE"]


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(r"D:\Ecommerce_Tableau\customer360_gold.csv")

# 1. Convert date column to numeric features
df['DT_CUSTOMER'] = pd.to_datetime(df['DT_CUSTOMER'])
df['CUSTOMER_MONTH'] = df['DT_CUSTOMER'].dt.month
df['CUSTOMER_YEAR'] = df['DT_CUSTOMER'].dt.year
df['CUSTOMER_DAY'] = df['DT_CUSTOMER'].dt.day
df.drop('DT_CUSTOMER', axis=1, inplace=True)

# 2. Label encode categorical columns
label_cols = ['EDUCATION', 'MARITAL_STATUS']

le = LabelEncoder()
for col in label_cols:
    df[col] = le.fit_transform(df[col])

# 3. Verify no object columns remain
print(df.dtypes)


ID                     int64
YEAR_BIRTH             int64
AGE                    int64
EDUCATION              int64
MARITAL_STATUS         int64
INCOME               float64
TOTAL_CHILDREN         int64
TOTAL_SPENT            int64
TOTAL_PURCHASES        int64
WEB_VISITS             int64
RECENCY                int64
CAMPAIGN_RESPONSE      int64
CUSTOMER_MONTH         int32
CUSTOMER_YEAR          int32
CUSTOMER_DAY           int32
dtype: object


In [8]:
from sklearn.model_selection import train_test_split

X = df.drop('CAMPAIGN_RESPONSE', axis=1)
y = df['CAMPAIGN_RESPONSE']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [9]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    enable_categorical=False
)

model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, ...)

In [10]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8504464285714286

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.95      0.92       379
           1       0.53      0.29      0.37        69

    accuracy                           0.85       448
   macro avg       0.70      0.62      0.64       448
weighted avg       0.83      0.85      0.83       448


Confusion Matrix:
 [[361  18]
 [ 49  20]]


In [11]:
df["PREDICTED_CHURN"] = model.predict(X)


In [12]:
df.to_csv("customer360_predictions.csv", index=False)
